<a href="https://colab.research.google.com/github/anasshaddad/datamatrix/blob/master/ProjetBrasIntelligent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import matplotlib.pyplot as plt
from keras_retinanet import models


In [ ]:
drive.mount('/content/drive')
get_ipython().system('unzip -uq "/content/drive/My Drive/Datamatrix.zip" -d "/content/drive/My Drive/D"')

path="/content/drive/My Drive/D/Datamatrix/train_labels.csv"
t=pd.read_csv(path)

In [ ]:
train_df, test_df = train_test_split(
 t,
  test_size=0.2,
 
)

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
CLASSES_FILE = 'classes.csv'
classes = set(['datamatrix'])

with open(CLASSES_FILE, 'w') as f:
  for i, line in enumerate(sorted(classes)):
    f.write('{},{}\n'.format(line,i))

ANNOTATIONS_FILE = 'annotations.csv'
train_df.to_csv(ANNOTATIONS_FILE, index=False, header=None)

In [ ]:
os.makedirs("snapshots", exist_ok=True)
PRETRAINED_MODEL = './snapshots/_pretrained_model.h5'

URL_MODEL = 'https://github.com/fizyr/keras-retinanet/releases/download/0.5.1/resnet50_coco_best_v2.1.0.h5'
urllib.request.urlretrieve(URL_MODEL, PRETRAINED_MODEL)

In [ ]:
get_ipython().system('keras_retinanet/bin/train.py'
--freeze-backbone  
--random-transform  
--weights {PRETRAINED_MODEL}  
--batch-size 1  --steps 60  
--epochs 50  
csv annotations.csv classes.csv)

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from pandas.plotting import register_matplotlib_converters
from sklearn.model_selection import train_test_split
import urllib
import os
import csv
import cv2
import time
from PIL import Image
import RPi.GPIO as GPIO
import time
from pylibdmtx import pylibdmtx


from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color
import RPi.GPIO as GPIO

In [ ]:


THRES_SCORE = 0.6 #accepter les prediction en haut de 60%

L1=56   #longeur bras 1
L2=70   #Longeur bras 2

Current_angle1 = 90
Current_angle2 = 0
steps=1.8     #steps of stepper motor Nemma 

x1=43  #position du tapis des carte PASS
y1=34
x2=245 #position du tapis des carte SCRAP
y2=475



delay = 0.2 #vitess de rotation du motor

widthpic=5   #largeur du field du camera
widthpixel=2179 #largeur du resoltuon du camera
heightpic=1303 #Hauteur du field du camera
heightpixel=2  #Hauteur du resolution du camera
Xcamera=0      #position du camera par rapport au repere du 1 moteur pas a pas
Ycamera=0

In [ ]:
ENA1 = 33 #moteur pas a pas 1
IN1 = 35 #Clock
IN2 = 37 #AntiClock

ENA2 = 20  #moteur pas a pas 2
IN3 = 21   #clock
IN4 = 22   #Anticlock

LS1 = 10   #limit switch 1
LS2 = 11   #limit switch 2
CP = 13 #Capteur de presence
svm = 12 #Servo moteur
Vaccum = 9 #Vacuum pump

In [ ]:
GPIO.setup(10,GPIO.OUT)
servo1 = GPIO.PWM(11,50) # pin 11 for servo1
GPIO.setup(12,GPIO.OUT)
servo2 = GPIO.PWM(12,50) # pin 12 for servo2
GPIO.setmode(GPIO.BOARD)

GPIO.setup(ENA1, GPIO.OUT, initial=GPIO.HIGH)#stepper motor 1
GPIO.setup(IN1, GPIO.OUT, initial=GPIO.LOW)
GPIO.setup(IN2, GPIO.OUT, initial=GPIO.LOW)

GPIO.setup(ENA2, GPIO.OUT, initial=GPIO.HIGH)#stepper motor 2
GPIO.setup(IN3, GPIO.OUT, initial=GPIO.LOW)
GPIO.setup(IN4, GPIO.OUT, initial=GPIO.LOW)

GPIO.setup(LS1, GPIO.IN, pull_up_down=GPIO.PUD_UP)#Limit switch
GPIO.setup(LS2, GPIO.IN, pull_up_down=GPIO.PUD_UP)#Limit switch
GPIO.setuo(CP, GPIO.IN,pull_up_down=GPIO.PUD_UP)#capteur de presence





state1 = GPIO.input(LS1)
state2 = GPIO.input(LS2)
capteurpre = GPIO.input(CP)

In [ ]:
model_path = r"C:\Users\manas\Desktop\Rapport PFE\resnet50_csv_50.h5"
print(model_path)

model = models.load_model(model_path, backbone_name='resnet50')
model = models.convert_model(model)

In [ ]:
def find_cintroid_pcb(image):
    img=cv2.cvtColor(image,cv2.COLOR_RGB2GRAY)

    ret,thresh = cv2.threshold(img,127,255,0)
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    c = max(contours, key = cv2.contourArea)
    M = cv2.moments(c)
    cx = int(M['m10']/M['m00'])
    cy = int(M['m01']/M['m00'])
    return cx,cy

In [ ]:
def initialiser():
    While(state1):
        GPIO.output(IN1, GPIO.LOW)
        GPIO.output(IN2, GPIO.HIGH)
        time.sleep(1)
    While(State2):
        GPIO.output(IN3, GPIO.LOW)
        GPIO.output(IN4, GPIO.HIGH)
        time.sleep(1)

In [ ]:
def Calculdesongles(x,y):
    
 if(x>0):  
    try:
        
         
    
             B=math.sqrt(x**2+y**2)
             q1 = math.atan2(y,x)
             q2 = math.acos((l1**2 + B**2 - l2**2)/(2*l1*B))
             Angle1= math.degrees(q1)+math.degrees(q2)
             Angle2 =180 - math.degrees(math.acos((l1**2+l2**2 - B**2 )/(2*l1*l2)))
             return(Angle1,Angle2)
    except:
        initialiser()
        
 else:
        initialiser()

In [ ]:
def angletosteps1(q):
    if(q != Current_angle1):
        if(q>0 and Current_angle1>0):
            if(q>Current_angle1):
                nsteps=(q-Current_angle1)/steps
                GPIO.output(IN1, GPIO.HIGH)
                Current_angle1=q
            if(q<Current_angle1):
                nsteps=(Current_angle1-q)/steps
                GPIO.output(IN2, GPIO.HIGH)
                Current_angle1=q
        if(q<0 and Current_angle1>0):
            nsteps=(abs(q)+Current_angle1)/steps
            GPIO.output(IN2, GPIO.HIGH)
            Current_angle1=q
        if(q>0 and Current_angle1<0):
            nsteps=(q+abs(Current_angle1))/steps
            GPIO.output(IN1, GPIO.HIGH)
            Current_angle1=q
        if(q<0 and Current_angle1<0):
            if(q>Current_angle1):
                nsteps=(abs(q)-abs(Current_angle1))/steps
                GPIO.output(IN1, GPIO.HIGH)
                Current_angle1=q
            if(q<Current_angle1):
                nsteps=(abs(Current_angle1)-abs(q))/steps
                GPIO.output(IN2, GPIO.HIGH)
                Current_angle1=q
    else:
        nsteps1=0
    return nsteps1

In [ ]:
def angletosteps2(q):
    if(q != Current_angle2):
        if(q>0 and Current_angle2>0):
            if(q>Current_angle2):
                nsteps=(q-Current_angle2)/steps
                GPIO.output(IN1, GPIO.HIGH)
                Current_angle2=q
            if(q<Current_angle2):
                nsteps=(Current_angle2-q)/steps
                GPIO.output(IN2, GPIO.HIGH)
                Current_angle2=q

        if(q<0 and Current_angle2>0):
            nsteps=(abs(q)+Current_angle2)/steps
            GPIO.output(IN2, GPIO.HIGH)
            Current_angle2=q
        if(q>0 and Current_angle2<0):
            nsteps=(q+abs(Current_angle2))/steps
            GPIO.output(IN1, GPIO.HIGH)
            Current_angle2=q
        if(q<0 and Current_angle2<0):
            if(q>Current_angle2):
                nsteps=(abs(q)-abs(Current_angle2))/steps
                GPIO.output(IN1, GPIO.HIGH)
                Current_angle2=q
            if(q<Current_angle2):
                nsteps=(abs(Current_angle2)-abs(q))/steps
                GPIO.output(IN2, GPIO.HIGH) 
                Current_angle2=q
    else:
        nsteps2=0
    return nsteps2

In [ ]:
def Turn_Motor1(nsteps):
    for x in range(0,nsteps):
        GPIO.output(EN1, GPIO.HIGH)
        sleep(delay)
        GPIO.output(EN1, GPIO.LOW)
        sleep(delay)

        


# In[ ]:


def Turn_Motor2(nsteps):
    for x in range(0,nsteps):
        GPIO.output(EN2, GPIO.HIGH)
        sleep(delay)
        GPIO.output(EN2, GPIO.LOW)
        sleep(delay)

In [ ]:
def changereper(x,y):
    x=x*widthpic/widthpixel
    y=y*widthpic/widthpixel
    x=Xcamera + x
    y=Ycamera + y 
    return(x,y)

In [ ]:
def predict(image):
  image = preprocess_image(image.copy())
  image, scale = resize_image(image)

  boxes, scores, labels = model.predict_on_batch(
    np.expand_dims(image, axis=0)
  )

  boxes /= scale

  return boxes, scores, labels

In [ ]:
def draw_detections(image, boxes, scores, labels):
  for box, score, label in zip(boxes[0], scores[0], labels[0]):
    if score < THRES_SCORE:
        break
    x,y=find_cintroid_pcb(image)
    (x,y)=changereper(x,y)
    processmouv(x,y)
    breathIn()
    b = box.astype(int)
    crop_img = image[b[1]:b[3],b[0]:b[2]]
    
    
    p=cv2.cvtColor(crop_img, cv2.COLOR_RGB2GRAY)
    
    msg = pylibdmtx.decode(p)
    data=msg
    Etat=CheckItac(data)
    if(Etat):
            processmouv(x1,y1)
            breathOut()
            
    else:
            processmouv(x2,y2)
            breathOut()
    
    color = [0,0,0]
   
    draw_box(image, b, color=color)

    caption = "{}".format(msg)
    draw_caption(image, b, caption)

In [ ]:
def gstreamer_pipeline(
    capture_width=3280,
    capture_height=2464,
    display_width=820,
    display_height=616,
    framerate=21,
    flip_method=0,
):
    return (
        "nvarguscamerasrc ! "
        "video/x-raw(memory:NVMM), "
        "width=(int)%d, height=(int)%d, "
        "format=(string)NV12, framerate=(fraction)%d/1 ! "
        "nvvidconv flip-method=%d ! "
        "video/x-raw, width=(int)%d, height=(int)%d, format=(string)BGRx ! "
        "videoconvert ! "
        "video/x-raw, format=(string)BGR ! appsink"
        % (
            capture_width,
            capture_height,
            framerate,
            flip_method,
            display_width,
            display_height,
        )
    )
if(!capteurpre):
    
    cap = cv2.VideoCapture(gstreamer_pipeline(), cv2.CAP_GSTREAMER)
    sleep(500)
    if cap.isOpened():
    
        ret, img = cap.read()
        cap.release()
    draw=show_detected_objects(img) #draw au cas ou on a voulu visualiser le resultat

In [ ]:
def processmouv(x,y):
    (teta1,teta2)=Calculdesongles(x,y)
    nstep1=angletosteps1(teta1)
    nstep2=angletosteps2(teta2)
    Turn_Motor1(nstep1)
    Turn_Motor2(nstep2)
    
    
    